## Day 19
You land in an airport surrounded by dense forest. As you walk to your high-speed train, the Elves at the Mythical Information Bureau contact you again. They think their satellite has collected an image of a sea monster! Unfortunately, the connection to the satellite is having problems, and many of the messages sent back from the satellite have been corrupted.

They sent you a list of the rules valid messages should obey and a list of received messages they've collected so far (your puzzle input).

The rules for valid messages (the top part of your puzzle input) are numbered and build upon each other. For example:

- 0: 1 2
- 1: "a"
- 2: 1 3 | 3 1
- 3: "b"

Some rules, like 3: "b", simply match a single character (in this case, b).

The remaining rules list the sub-rules that must be followed; for example, the rule 0: 1 2 means that to match rule 0, the text being checked must match rule 1, and the text after the part that matched rule 1 must then match rule 2.

Some of the rules have multiple lists of sub-rules separated by a pipe (|). This means that at least one list of sub-rules must match. (The ones that match might be different each time the rule is encountered.) For example, the rule 2: 1 3 | 3 1 means that to match rule 2, the text being checked must match rule 1 followed by rule 3 or it must match rule 3 followed by rule 1.

Fortunately, there are no loops in the rules, so the list of possible matches will be finite. Since rule 1 matches a and rule 3 matches b, rule 2 matches either ab or ba. Therefore, rule 0 matches aab or aba.

Here's a more interesting example:

- 0: 4 1 5
- 1: 2 3 | 3 2
- 2: 4 4 | 5 5
- 3: 4 5 | 5 4
- 4: "a"
- 5: "b"

Here, because rule 4 matches a and rule 5 matches b, rule 2 matches two letters that are the same (aa or bb), and rule 3 matches two letters that are different (ab or ba).

Since rule 1 matches rules 2 and 3 once each in either order, it must match two pairs of letters, one pair with matching letters and one pair with different letters. This leaves eight possibilities: aaab, aaba, bbab, bbba, abaa, abbb, baaa, or babb.

Rule 0, therefore, matches a (rule 4), then any of the eight options from rule 1, then b (rule 5): aaaabb, aaabab, abbabb, abbbab, aabaab, aabbbb, abaaab, or ababbb.

The received messages (the bottom part of your puzzle input) need to be checked against the rules so you can determine which are valid and which are corrupted. Including the rules and the messages together, this might look like:

- 0: 4 1 5
- 1: 2 3 | 3 2
- 2: 4 4 | 5 5
- 3: 4 5 | 5 4
- 4: "a"
- 5: "b"

- ababbb
- bababa
- abbbab
- aaabbb
- aaaabbb

Your goal is to determine the number of messages that completely match rule 0. In the above example, ababbb and abbbab match, but bababa, aaabbb, and aaaabbb do not, producing the answer 2. The whole message must match all of rule 0; there can't be extra unmatched characters in the message. (For example, aaaabbb might appear to match rule 0 above, but it has an extra unmatched b on the end.)

How many messages completely match rule 0?

In [1]:
import numpy as np
import regex

In [2]:
# Open file
f = open("input_data/Day19.txt", "r")

# Read in the rules
rules = {}

line=f.readline()
while (line != '\n'):
    line = line.strip('\n')
    rule_no, rule = line.split(':')
    rules[int(rule_no)] = rule.strip()
    line = f.readline()
    
# Clean up terminating rules
rules[91] = 'b'
rules[20] = 'a'

# Read in the test strings

messages = []

line=f.readline()
while line:
    line = line.strip('\n')
    messages.append(line)
    line = f.readline()
    
# Close file
f.close()

In [3]:
def process_rule(rule_string):
    '''recursively processes a rule, building up a string that is a huge regular expression'''
    
    # initialize our regular expression
    reg_exp = ''
    
    # first we look for a termination point
    if rule_string in ['a','b']:
        return rule_string
    
    # otherwise we break rule_string into its parts
    rule_list = rule_string.split()
    
    # Look at each item in the list
    reg_exp = '('
    for r in rule_list:
        
        # if we have a pipe add it to our regular expression
        if r == '|':
            reg_exp += r
        else:
            reg_exp += process_rule(rules[int(r)])
    
    return reg_exp + ')'

In [4]:
reg_exp = process_rule('0')
print(reg_exp)

((((a((b(b(a((ba)a|(ba|a(b|a))b)|b(b(ba)|a(bb)))|a(b(a(ba)|b(aa))|a((aa|(b|a)b)b|(ba|(b|a)b)a)))|a(a(((b(b|a)|aa)(b|a))b|((aa|(b|a)b)b|(ba|(b|a)b)a)a)|b(((aa|(b|a)b)b|(ba|a(b|a))a)b|((ba)a|(b(b|a)|aa)b)a)))a|((a(((b|a)(aa|bb))a|((ba|a(b|a))(b|a))b)|b(((ba|a(b|a))b|(ab|aa)a)b|(a(aa|(b|a)b))a))b|(((b(ab|ba)|a(aa|bb))a|((ba)a|(ba|a(b|a))b)b)a|(((aa|ba)b|(aa)a)b|((ba|a(b|a))b|(aa)a)a)b)a)b)|b(b((b(b((ba)a|(ba|a(b|a))b)|a((bb)b|(bb)a))|a(a(a(aa|ba)|b(ba|a(b|a)))|b(a(aa|(b|a)b))))a|(((b(ba|(b|a)b)|a(ba|a(b|a)))b|(a(aa|bb)|b(bb|ba))a)a|(b(a(bb))|a(b(bb)|a(bb)))b)b)|a(((b((ab|ba)b|(aa)a)|a((b|a)(ba|a(b|a))))a|(((ba)a|(b(b|a)|aa)b)a|((bb|ba)b)b)b)b|((((b|a)(ba|(b|a)b))a|(b(ba))b)a|(a((ab|ba)b|(ba|a(b|a))a)|b((ab|ba)b|(b(b|a)|aa)a))b)a))))((a((b(b(a((ba)a|(ba|a(b|a))b)|b(b(ba)|a(bb)))|a(b(a(ba)|b(aa))|a((aa|(b|a)b)b|(ba|(b|a)b)a)))|a(a(((b(b|a)|aa)(b|a))b|((aa|(b|a)b)b|(ba|(b|a)b)a)a)|b(((aa|(b|a)b)b|(ba|a(b|a))a)b|((ba)a|(b(b|a)|aa)b)a)))a|((a(((b|a)(aa|bb))a|((ba|a(b|a))(b|a))b)|b(((ba|a(b|a))

In [5]:
# now let's compile our regular expression
p = regex.compile(reg_exp)

# initialize our sum of matches
matches = 0

# And now we loop through our messages counting matches
for message in messages:
    
    m = p.match(message)
    if m and (m.start()==0) and (m.end()==len(message)):
        matches += 1

print('Number of matches:', matches)

Number of matches: 195


### Part 2

In [108]:
# Let's stay with our original data and see what rules 8, 11, 31, and 42 generate for a regular expression

reg_exp_8  = process_rule('8')
reg_exp_11 = process_rule('11')
reg_exp_31 = process_rule('31')
reg_exp_42 = process_rule('42')

# Strip leading and trailing parentheses
reg_exp_8 = reg_exp_8[1:len(reg_exp_8)-1]
reg_exp_11 = reg_exp_11[1:len(reg_exp_11)-1]
reg_exp_31 = reg_exp_31[1:len(reg_exp_31)-1]
reg_exp_42 = reg_exp_42[1:len(reg_exp_42)-1]

In [112]:
# New rules are:
# 8: 42 | 42 8
# 11: 42 31 | 42 11 31

# Try this with recursion
new_reg_exp_8 = '(' + reg_exp_42 + ')+'

matches = 0
for i in range(1,9):
    new_reg_exp_11 = '(' + i*reg_exp_42 + i*reg_exp_31 + ')'
    
    # Rule 0: 8 11 )
    p2_reg_exp = '(' + new_reg_exp_8 + ')(' + new_reg_exp_11 + ')'
    
    p2_p = regex.compile(p2_reg_exp)

    # And now we loop through our messages counting matches
    for message in messages:
    
        m = p2_p.match(message)
        if m and (m.start()==0) and (m.end()==len(message)):
            matches += 1

print('Number of matches:', matches)

Number of matches: 309
